# Introduction

## Dataset Description
The dataset that is being worked on is called Flight Delay, and has significant amounts of information on flights from the years 2018-2024. This set of data has 29 columns, with information relating to date of flight, overall flight time, location data, and delay explanations.

The dataset can be found here: https://www.kaggle.com/datasets/arvindnagaonkar/flight-delay/data

## Proposed Question(s)

Is there a correlation between domestic flight delays and weather in the domestic airline market?

# Data Exploration

In [ ]:
# Import Packages
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [ ]:
# Read parquet file, make df. Source: https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe
df = pd.read_parquet('Flight_Delay.parquet', engine='pyarrow')

In [ ]:
# Check layout of dataset visually.
df.head()

In [ ]:
# Check for any abnormal information, column types.
df.info()

In [ ]:
# Check for null values in dataset.
df.isnull().sum()

In [ ]:
# Check for duplicated values.
df.duplicated().sum()

In [ ]:
# Check min and max in WeatherDelay column for skewed data.
print(df.WeatherDelay.max())
print(df.WeatherDelay.min())

In [ ]:
# Check departure city locations.
# Source: https://www.geeksforgeeks.org/get-unique-values-from-a-column-in-pandas-dataframe/
df.OriginCityName.unique()

In [ ]:
# Check destination city names. 
# Source: https://www.geeksforgeeks.org/get-unique-values-from-a-column-in-pandas-dataframe/
df.DestCityName.unique()

In [ ]:
# Check years.
# Source: https://www.geeksforgeeks.org/get-unique-values-from-a-column-in-pandas-dataframe/
df.Year.unique()

In [ ]:
# Initial simple scatter plot visual based on year.
plt.scatter(df.Year, df.WeatherDelay)

# Data Cleaning

Initial dataset is very clean, and only minor adjustmetns are needed for the analysis.

Based on the above information these steps will be performed:

Columns 'DayofMonth', 'Marketing_Airline_Network', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelayMinutes', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Distance', 'DistanceGroup' will be removed, as they will not be used in analysis.

Origin City and Departure City will have their States split off into Origin State and Departure State for possilbe use in data analysis.

Floats will be converted to ints. This will be done to simplify the analysis.

New column CleanDates will be created. This column will be used in the regression model, as the Flight Date Column will not be readable by the model.

## Column Removal

Removal of Columns 'Month', 'DayofMonth', 'Marketing_Airline_Network', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelayMinutes', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Distance', 'DistanceGroup'.

In [ ]:
# Check columns for removal.
df.info()

In [ ]:
# Drop columns, asign to new dataframe.
df_clean = df.drop(['Month', 'DayofMonth', 'Marketing_Airline_Network', 'CRSDepTime', 'DepTime', 'DepDelayMinutes',
                    'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelayMinutes',
                    'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Distance', 'DistanceGroup'], axis=1)

In [ ]:
# Check for dropped columns.
df_clean.info()

## Column Splitting

Splitting off of Origin City and Departure City into state and city columns.

In [ ]:
# Split OriginCityName, assign to split. Source: https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/
split = df_clean.OriginCityName.str.split(', ', n=1, expand=True)

In [ ]:
# Check data.
split.head()

In [ ]:
# Create two new columns with the split data.
# Source: https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/
df_clean['OriginCity'] = split[0]
df_clean['OriginState'] = split[1]

In [ ]:
# Drop the orignal column.
df_clean.drop(columns='OriginCityName', inplace=True)

In [ ]:
# Split DestCityName, assign to split.
# Source: https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/
split = df_clean.DestCityName.str.split(', ', n=1, expand=True)

In [ ]:
# Check data.
split.head()

In [ ]:
# Create two new columns with the split data.
# Source: https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/
df_clean['DestCity'] = split[0]
df_clean['DestState'] = split[1]

In [ ]:
# Drop the orignal column.
df_clean.drop(columns='DestCityName', inplace=True)

In [ ]:
# Check that columns were dropped, new columns exist.
df_clean.info()

In [ ]:
# Check origin city names. 
# Source: https://www.geeksforgeeks.org/get-unique-values-from-a-column-in-pandas-dataframe/
df_clean.OriginCity.unique()

In [ ]:
# Check origin state names. 
# Source: https://www.geeksforgeeks.org/get-unique-values-from-a-column-in-pandas-dataframe/
df_clean.OriginState.unique()

In [ ]:
# Check destination city names. 
# Source: https://www.geeksforgeeks.org/get-unique-values-from-a-column-in-pandas-dataframe/
df_clean.DestCity.unique()

In [ ]:
# Check destination state names. 
# Source: https://www.geeksforgeeks.org/get-unique-values-from-a-column-in-pandas-dataframe/
df_clean.DestState.unique()

## Column Conversion

Conversion of float columns to int columns.

In [ ]:
# Check columns to convert.
df_clean.info()

In [ ]:
# Convert float column to int column. 
# Source: https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
df_clean = df_clean.astype({"DepDelay": 'int64', "ArrDelay": 'int64', "CarrierDelay": 'int64', "WeatherDelay": 'int64', "NASDelay": 'int64',
                            "SecurityDelay": 'int64', "LateAircraftDelay": 'int64'})

In [ ]:
# Check that columns were changed.
df_clean.info()

In [ ]:
# Check values in dataset.
df_clean.head()

## Create Column CleanDate

The column FlightDate will be converted into a format that is readable by the regression model.

In [ ]:
# Convert dates to numbers for regression model.
# Source: https://www.youtube.com/watch?v=bmFVs5XkUbM
clean_dates = mdates.date2num(df_clean.FlightDate)

In [ ]:
# Create CleanDates, add to dataframe.
# Source: https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/
df_clean['CleanDates'] = clean_dates
# Convert float column to int column. Source: https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
df_clean = df_clean.astype({"CleanDates": 'int64'})

In [ ]:
# Check for column, check that it's type int.
df_clean.info()

In [ ]:
# Visually check column.
df_clean.head()

In [ ]:
# Save cleaned df to csv. Source: https://www.geeksforgeeks.org/saving-a-pandas-dataframe-as-a-csv/
df_clean.to_csv('flight_delay_clean.csv')

# Data Analysis

## Question: Is there a correlation between domestic flight delays and weather in the domestic airline market?

We will be using the pearson r regression model. The column FlightDate will be variable x. WeatherDelay will be variable y.

In [ ]:
# Assign columns to X and y.
X = df_clean.CleanDates
y = df_clean.WeatherDelay

In [ ]:
# Source: https://realpython.com/numpy-scipy-pandas-correlation-python/#linear-correlation
# Source: https://www.youtube.com/watch?v=bmFVs5XkUbM
# Source: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
stats.pearsonr(X, y)

Based on the above results, the null hypothesis can be rejected, but the statistic results in minimal to no correlation.

# Data 

While the statistical test shows minimal corelation, we can still visually represent the results for the final report.